In [1]:
%run ./src/__init__.py

from time import process_time

In [9]:
## Get artists dataframe and asign id

df_artists = pd.read_csv('./data/artist_uris.csv', header=None, names=['name', 'uri'])

df_artists[['uri1','uri2','id']] = df_artists.uri.str.split(':',3,expand=True)
df_artists.drop(columns=['uri1','uri2'], inplace=True)

df_artists['genres'] = ''
df_artists['popularity'] = 0

In [10]:
## Get artists genres

t_start = process_time()

for i in range(0,81350,50):
    ids = df_artists[i:i+49]['id']
    features = sp.artists(ids)['artists']
    for feature, j in zip(features,range(i,i+49)):
        df_artists['genres'][j] = '|'.join(feature['genres'])
        df_artists['popularity'][j] = feature['popularity']
    
t_end = process_time()
print('Duration:',t_end-t_start)

<ipython-input-10-fb600786421a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artists['genres'][j] = '|'.join(feature['genres'])
<ipython-input-10-fb600786421a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artists['popularity'][j] = feature['popularity']


Duration: 356.901033928


In [11]:
## % of artists without genres

len(df_artists[df_artists.genres==''])/len(df_artists) *100

42.93373338415946

In [12]:
## Filter artists with genres

df_artists_filt = df_artists[df_artists.genres!='']

df_artists_filt.reset_index(drop=True, inplace=True)

In [14]:
## get seed genres

genres = pd.read_csv('./data/seed_genres_clean.csv')

genres_list = genres['seed_genre'].drop_duplicates().to_list()
genres_list.sort()

In [15]:
## prepare genres columns

for genre in genres_list:
    df_artists_filt[genre] = 0

<ipython-input-15-f111b45b91a0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artists_filt[genre] = 0


In [16]:
## dictionary of gen

seed_genre_dict = pd.Series(genres.seed_genre.values,index=genres.genre).to_dict()

In [17]:
def get_seed_genre(gen):
    if gen in seed_genre_dict.keys():
        return seed_genre_dict[gen]
    else:
        return 'unmapped'
    
df_artists_filt['unmapped'] = 0

for i in range(len(df_artists_filt)):
    gen_set = set([get_seed_genre(gen) for gen in df_artists_filt.genres[i].split('|')])
    for gen in gen_set:
        df_artists_filt[gen][i] = 1

<ipython-input-17-4baaf54706b4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artists_filt['unmapped'] = 0
<ipython-input-17-4baaf54706b4>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artists_filt[gen][i] = 1
/home/sergiogg/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.u

In [18]:
df_sum = df_artists_filt.sum(numeric_only = True, axis=1)

df_sum[df_sum==0]

Series([], dtype: int64)

In [19]:
df_artists_filt

,name,uri,id,genres,popularity,acoustic,afrobeat,afrobeatbeat,alternative,ambient,...,sleep,soul,study,tango,techno,trance,turkish,vocal,world-music,unmapped
0,1:43,spotify:artist:39EHxSQAIaWusRqSI9xoyF,39EHxSQAIaWusRqSI9xoyF,pinoy idol pop,35,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2:00 AM,spotify:artist:4tN3rZ7cChj4Wns2Wt2Nj6,4tN3rZ7cChj4Wns2Wt2Nj6,antiviral pop|k-pop|minecraft,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2:54,spotify:artist:3LsQKoRgMc8VEkQn66jfAQ,3LsQKoRgMc8VEkQn66jfAQ,metropopolis,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6:33,spotify:artist:0oBPg2seHzVcAIOpdi1Ojj,0oBPg2seHzVcAIOpdi1Ojj,avant-garde metal,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,spotify:artist:1BYnP5HY7Ci0MFEHiDV1Ww,1BYnP5HY7Ci0MFEHiDV1Ww,neo-progressive,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46403,黃小琥,spotify:artist:6KCusBln9NTESgcuI0DlUz,6KCusBln9NTESgcuI0DlUz,c-pop|classic mandopop|mandopop|taiwan pop,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46404,黃曉明,spotify:artist:53F8atvCmVFVOvnKwZXBd3,53F8atvCmVFVOvnKwZXBd3,mainland chinese pop,20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46405,黃玠瑋,spotify:artist:6VI0p0xTjBKKxQN8i8vGpD,6VI0p0xTjBKKxQN8i8vGpD,chinese electropop|chinese indie|taiwan indie|...,43,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46406,黃義達,spotify:artist:7kaq0LysuRSgBZSorlZ7Vj,7kaq0LysuRSgBZSorlZ7Vj,mandopop|singaporean mandopop,39,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
## Save dataframe

df_artists_filt.to_csv('./data/artist_genres.csv', header=True, index=False)